In [3]:
!pip install stanza
import pandas as pd
import stanza
import re


stanza.download('es', verbose=False)
es_nlp = stanza.Pipeline('es', processors='tokenize,lemma,pos,depparse', verbose=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=3371e41bb598c70bc961df9f069fd57675be0527e0eb33514e95c1bc878a8152
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


*PROGRAMA* Limpiar el data frame y completar los Nan

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ATENCION_POSTVENTAS.csv')
df.drop(['Section/Column','Start Date', 'Due Date',
       'Blocked By (Dependencies)', 'Blocking (Dependencies)',
       'Revisión garantía', 'Recibidos', 'Parent Task', 'Assignee',
       'Assignee Email', 'Name', 'Tags', 'Projects', 'Fecha',
       'Dirección correo','Satisfacción', 'Foto de trabajo realizado.'],
        axis=1, inplace=True)

df = df[df['Notes'].notna()]
df.reset_index(drop=True, inplace=True)

percent_missing = df.isnull().sum() * 100 / len(df)
print(percent_missing)
print(df.index)



Task ID           0.000000
Created At        0.000000
Completed At      1.773050
Last Modified     0.000000
Notes             0.000000
Nombre           47.163121
Casa             47.163121
Proyecto         47.163121
Manzana          72.695035
Télefono         57.446809
Solicitud 1      81.560284
Solicitud 2      90.425532
Solicitud 3      95.744681
Solicitud 4      97.163121
dtype: float64
RangeIndex(start=0, stop=282, step=1)


Usa esto para encontrar las palabras que se necesitan para homogeinizar la información.

In [ ]:
# This part would be re writed in julia (coming soon)
from collections import Counter

list_to_set = []
data_proc = (df['Notes'].apply(lambda x: x.split('\n')))
for note in data_proc:
    for word in note:
        list_to_set.append(word)

Counter = Counter(list_to_set)
print(Counter.most_common(20))

[('', 2376), ('Nombre:', 281), ('Dirección de email:', 281), ('———————————————', 281), ('Numero de contacto:', 276), ('Proyecto:', 276), ('Solicitud 1:', 276), ('This task was submitted through SOLICITUD DE POSTVENTAS Y GARANTIAS ', 276), ('https://form.asana.com/?k=NUy_diulYpY4CoQVB7TU4A&d=937223421497983', 265), ('Armonía', 218), ('Fecha de solicitud:', 152), ('Dirección:', 150), ('Solicitud 2:', 133), ('Manzana::', 130), ('Número de casa::', 126), ('Fecha de solicitud::', 122), ('Solicitud 3:', 62), ('Fortaleza', 43), ('Solicitud 4:', 33), ('6', 19)]


De la información anterior tenemos que existe algunas que tienen la dirección unida y otras que tienen la dirección separada. Para este caso lo mejor será dividir la dirección que está unida. Pues es importante las variables manzana y también casa.

Usa esto para probar los modelos de rellenar datos, antes de utilizarlos en la base de datos

In [ ]:
n = 0
prob = df['Notes'].iloc[n]
print(df['Notes'].iloc[n])

Nombre:
Hedlyn Antonio castrillon García

Dirección de email:
cas.trillon@hotmail.com

Numero de contacto:
3135025994

Proyecto:
Cassia

Número de casa::
32

Manzana::
1

Fecha de solicitud::
Jan 30, 2023

Solicitud 1:
El agua no sube al segundo piso


Solicitud 2:
Yesos caídos en paredes 


Solicitud 3:
Tubería del lava platos mal estado con fuga

———————————————
This task was submitted through SOLICITUD DE POSTVENTAS Y GARANTIAS 
https://form.asana.com/?k=NUy_diulYpY4CoQVB7TU4A&d=937223421497983


Para rellenar la información faltante de casas y manzana C:

In [ ]:
import re

def wow(note, regx, regx_b):    
    if regx != '(^c.*sa$|^k.*sa$)' and regx != '(^mz$|^m.*z.*a$|^maz$)':
        try:
            note = note.split('\n')
            note = [part.lower().replace(':', '') for part in note]
            search = list(re.match(regx, part) for part in note)
            bol_index = [True if part != None else False for part in search].index(True)
            return note[bol_index + 1]
        except:
            print(note)

    else:
        note = note.split()
        note = [part.lower().replace(':', '') for part in note]
        try:   
            search = list(re.match(regx, part) for part in note)
            bol_index = [True if part != None else False for part in search].index(True)

            # when client put redundant info
            if re.match(regx, note[bol_index + 1]) != None:
                return note[bol_index + 2] 

        # when client invert comon inputs... but spicify the mistake
            elif re.match(regx_b, note[bol_index + 1]) != None:
                search = list(re.match(regx, part) for part in note)
                bol_index = [True if part != None else False for part in search].index(True, bol_index + 1)
                return note[bol_index + 1] 
            else:
                return note[bol_index + 1]

        except: # this clients surpass me! XD
            return 'Necesario buscar en la base de datos'

print('block:')
print(wow(prob, '(^mz$|^m.*z.*a$|^maz$)', '(^c.*sa$|^k.*sa$)'))
print('house:')
print(wow(prob, '(^c.*sa$|^k.*sa$)', '(^mz$|^m.*z.*a$|^maz$)'))
print('name:')
print(wow(prob, 'nombre', 'asdfasdg'))
print('email')
print(wow(prob, '.*email', 'asdfas'))
print('phone:')
print(wow(prob, '.*contacto', 'asdfasdf'))
print('project:')
print(wow(prob, '.*proyecto', 'asdfasdf'))

block:
1
house:
32
name:
hedlyn antonio castrillon garcía
email
cas.trillon@hotmail.com
phone:
3135025994
project:
cassia


In [ ]:
import collections
# This is very slow, it would be changed in the future to julia

def warr(note):
    list_words = []
    note = note.lower().replace(':', '')
    start = 'solicitud 1'
    end = '———————————————'
    name_ = note[note.find(start):note.rfind(end)]
    namex = es_nlp(name_)
    for sentence in namex.sentences:
        for token in sentence.tokens:
            for word in token.words:
                if word.upos in ['NOUN', 'ADJ', 'VBD'] and word.text != 'solicitud':
                   list_words.append(word.lemma)
    list_words = ' ,'.join(set(list_words))
    return pd.Series([name_.replace('\n', ' '), list_words])

warr(prob)

0    solicitud 1 el agua no sube al segundo piso   ...
1    mal ,piso ,yeso ,plato ,pared ,segundo ,lava ,...
dtype: object

In [ ]:
df['name'] = df['Notes'].apply(lambda x: wow(x, 'nombre', 'asdfasdg'))
df['email'] = df['Notes'].apply(lambda x: wow(x, '.*email', 'asdfasdg'))
df['phone'] = df['Notes'].apply(lambda x: wow(x, '.*contacto', 'asdfasdf'))
df['project'] = df['Notes'].apply(lambda x: wow(x, '.*proyecto', 'asdfasdf'))
df['house'] = df['Notes'].apply(lambda x: wow(x, '(^c.*sa$|^k.*sa$)', '(^mz$|^m.*z.*a$|^maz$)'))
df['block'] = df['Notes'].apply(lambda x: wow(x, '(^mz$|^m.*z.*a$|^maz$)', '(^c.*sa$|^k.*sa$)'))
df[['warranty issues', 'key words']]=df['Notes'].apply(warr)

['el cliente manifiesta haber dejado la llave de la ducha en la noche abierta el agua reboso el nivel de la ducha y se filtro a habitación y placa de la sala ', '- se realizará sondeo de sifón de ducha para descartar taponamientos ']
['el cliente manifiesta haber dejado la llave de la ducha en la noche abierta el agua reboso el nivel de la ducha y se filtro a habitación y placa de la sala ', '- se realizará sondeo de sifón de ducha para descartar taponamientos ']
['nombre', 'armando enrique miranda pertuz ', '', 'dirección de email', 'armandoenriquemirandapertuz@gmail.com ', '', 'número de teléfono', '3006339717', '', 'proyecto', 'armonía', '', 'casa', '8', '', 'manzana', '4', '', 'recibido de trabajo 1', 'si rebido', '', '¿qué tan satisfecho está con la garantía prestada?', 'satisfecho', '', 'algún otro comentario', 'mejoramiento el servio de agua tengo problemas en ese asunto ', '', '———————————————', 'this task was submitted through satisfacción de postventa.', 'https//form.asana.co

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
print(percent_missing)
print(df.index)

Task ID             0.000000
Created At          0.000000
Completed At        1.773050
Last Modified       0.000000
Notes               0.000000
Nombre             47.163121
Casa               47.163121
Proyecto           47.163121
Manzana            72.695035
Télefono           57.446809
Solicitud 1        81.560284
Solicitud 2        90.425532
Solicitud 3        95.744681
Solicitud 4        97.163121
name                0.354610
email               0.354610
phone               1.063830
project             0.354610
house               0.000000
block               0.000000
warranty issues     0.000000
key words           0.000000
dtype: float64
RangeIndex(start=0, stop=282, step=1)


In [ ]:
df = df[['Task ID','Created At', 'Last Modified', 'project', 'block', 'house', 'name', 'email', 'phone', 'warranty issues', 'key words']]
df = df.dropna(axis=0)
percent_missing = df.isnull().sum() * 100 / len(df)
print(percent_missing)
print(df.index)

Task ID            0.0
Created At         0.0
Last Modified      0.0
project            0.0
block              0.0
house              0.0
name               0.0
email              0.0
phone              0.0
warranty issues    0.0
key words          0.0
dtype: float64
Int64Index([  0,   1,   2,   3,   4,   5,   6,   9,  10,  11,
            ...
            272, 273, 274, 275, 276, 277, 278, 279, 280, 281],
           dtype='int64', length=279)


In [4]:
# CHECKPOINT the data is not raw anymore, but still need some proccessing
#df.to_csv('posventas_procesado.csv')
#df.to_excel('posventas_procesado.xlsx')

dfp = pd.read_csv('/content/drive/MyDrive/posventas_procesado.csv')
print(dfp)

     Unnamed: 0  Unnamed: 0.1           Task ID  Created At Last Modified  \
0             0             0  1203859132730413  2023-01-30    2023-02-02   
1             1             1  1203827119997540  2023-01-25    2023-02-02   
2             2             2  1203823671025556  2023-01-24    2023-02-01   
3             3             3  1203713756246383  2023-01-12    2023-01-23   
4             4             4  1203695543348748  2023-01-10    2023-01-23   
..          ...           ...               ...         ...           ...   
274         274           277  1201002776765269  2021-09-16    2021-09-24   
275         275           278  1200915414083089  2021-09-03    2021-09-17   
276         276           279  1200830708641915  2021-08-21    2021-09-17   
277         277           280  1200788935337311  2021-08-15    2021-08-26   
278         278           281  1200788789910526  2021-08-15    2021-08-26   

        project block  house                              name  \
0        

Block and house still need some work, need to erease the 0 at begin of the numbers and some data still have errors

In [7]:
#dfp['house'] = dfp['house'].apply(lambda x:''.join(char for char in x if char.isdigit()))
dfp['block'] = dfp['block'].apply(lambda x: x[1:] if x[0] == '0' else x)
#dfp['block'] = dfp['block'].apply(lambda x: re.sub(''))



In [8]:
print(dfp['house'].tolist())
print(dfp['block'].tolist())
dfp.to_csv('/content/drive/MyDrive/posventas_procesado.csv')

[32.0, 8.0, 8.0, 26.0, 8.0, 3.0, 5.0, 21.0, 17.0, 5.0, 34.0, 8.0, 1.0, 18.0, 1.0, 23.0, 26.0, 15.0, 23.0, 24.0, nan, 26.0, 34.0, 12.0, nan, 8.0, nan, nan, 9.0, 26.0, nan, 10.0, 24.0, 26.0, nan, 26.0, 26.0, 33.0, 1.0, 6.0, 17.0, 18.0, 7.0, 6.0, 24.0, 32.0, 7.0, 7.0, 30.0, 16.0, 18.0, 15.0, 30.0, 17.0, 18.0, 30.0, 1.0, nan, 16.0, 16.0, 4.0, nan, 17.0, 18.0, 29.0, 25.0, 9.0, 11.0, nan, 18.0, 1.0, 7.0, 2.0, 1.0, nan, 14.0, 13.0, 17.0, 8.0, 26.0, 17.0, 6.0, 17.0, 17.0, 25.0, nan, 10.0, 3.0, 13.0, 10.0, 10.0, 14.0, 1.0, 15.0, 1.0, 10.0, 4.0, 11.0, 1.0, 5.0, 15.0, 9.0, 16.0, 17.0, 32.0, 2.0, 8.0, 7.0, 18.0, 12.0, nan, 7.0, 7.0, 6.0, 7.0, 14.0, 4.0, 14.0, 10.0, 21.0, 18.0, 6.0, 6.0, 5.0, 2.0, 28.0, 11.0, 29.0, 5.0, 2.0, 6.0, 4.0, 24.0, 8.0, 29.0, 22.0, 6.0, 10.0, 25.0, 20.0, 22.0, 7.0, 3.0, 16.0, 14.0, 27.0, 14.0, 20.0, 24.0, 18.0, 21.0, 26.0, 22.0, 26.0, 30.0, 18.0, 6.0, 15.0, 9.0, 10.0, 29.0, 11.0, 26.0, 23.0, 6.0, 12.0, 6.0, 12.0, 25.0, 2.0, 13.0, 10.0, 21.0, 26.0, 23.0, 7.0, 18.0, 24.0, 13

In [28]:

dfp2 = dfp.groupby('name')['house', 'block'].apply(lambda x: x.ffill().bfill())
dfp.loc[:,['house', 'block']] = dfp2.loc[:,['house', 'block']]
dfp.drop_duplicates(inplace=True)
print(dfp)
dfp.to_csv('atencion_procc_posventas.csv')

     Unnamed: 0  Unnamed: 0.1           Task ID  Created At Last Modified  \
0             0             0  1203859132730413  2023-01-30    2023-02-02   
1             1             1  1203827119997540  2023-01-25    2023-02-02   
2             2             2  1203823671025556  2023-01-24    2023-02-01   
3             3             3  1203713756246383  2023-01-12    2023-01-23   
4             4             4  1203695543348748  2023-01-10    2023-01-23   
..          ...           ...               ...         ...           ...   
274         274           277  1201002776765269  2021-09-16    2021-09-24   
275         275           278  1200915414083089  2021-09-03    2021-09-17   
276         276           279  1200830708641915  2021-08-21    2021-09-17   
277         277           280  1200788935337311  2021-08-15    2021-08-26   
278         278           281  1200788789910526  2021-08-15    2021-08-26   

        project block  house                              name  \
0        

<ipython-input-28-1d709cc508f0>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfp2 = dfp.groupby('name')['house', 'block'].apply(lambda x: x.ffill().bfill())


In [31]:
db = pd.read_csv('/content/drive/MyDrive/Ctvid/db_posventa.csv')
db.sort_values('name', inplace=True)
print(db)
db.to_csv('/content/drive/MyDrive/Ctvid/db_posventa.csv')

     Unnamed: 0           Task ID  Created At Last Modified    project block  \
239         243  1201712665647044  2022-01-24    2022-01-27    armonía     2   
253         257  1201610229126161  2022-01-02    2022-01-27    armonía     3   
199         203  1202006846774044  2022-03-22    2022-07-18    armonía     3   
200         204  1201960931271746  2022-03-13    2022-04-22    armonía     3   
149         153  1202675201398351  2022-07-27    2022-11-09    armonía     6   
..          ...               ...         ...           ...        ...   ...   
13           16  1202295623989282  2022-05-17    2022-11-05  fortaleza     a   
18           21  1202157862044855  2022-04-20    2022-05-03  fortaleza     a   
32           35  1201536914335332  2021-12-19    2023-02-08  fortaleza     a   
30           33  1201668542314793  2022-01-15    2022-01-19  fortaleza     a   
33           36  1201453249574127  2021-12-01    2021-12-23  fortaleza     a   

     house                          nam